In [1]:
# Adapted from
# https://github.com/keras-team/keras/blob/master/examples/addition_rnn.py

In [2]:
import warnings
warnings.filterwarnings('ignore')

In [3]:
%matplotlib inline
%pylab inline

Populating the interactive namespace from numpy and matplotlib


In [4]:
import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
print(tf.__version__)

1.8.0


### Step 1: Generate sample equations

In [5]:
class CharacterTable(object):
    """Given a set of characters:
    + Encode them to a one hot integer representation
    + Decode the one hot integer representation to their character output
    + Decode a vector of probabilities to their character output
    """
    def __init__(self, chars):
        """Initialize character table.

        # Arguments
            chars: Characters that can appear in the input.
        """
        self.chars = sorted(set(chars))
        self.char_indices = dict((c, i) for i, c in enumerate(self.chars))
        self.indices_char = dict((i, c) for i, c in enumerate(self.chars))

    def encode(self, C, num_rows):
        """One hot encode given string C.

        # Arguments
            num_rows: Number of rows in the returned one hot encoding. This is
                used to keep the # of rows for each data the same.
        """
        x = np.zeros((num_rows, len(self.chars)))
        for i, c in enumerate(C):
            x[i, self.char_indices[c]] = 1
        return x

    def decode(self, x, calc_argmax=True):
        if calc_argmax:
            x = x.argmax(axis=-1)
        return ''.join(self.indices_char[x] for x in x)


In [6]:
class colors:
    ok = '\033[92m'
    fail = '\033[91m'
    close = '\033[0m'

In [7]:
# Parameters for the model and dataset.
TRAINING_SIZE = 50000
DIGITS = 3
# REVERSE = True
REVERSE = False

# Maximum length of input is 'int + int' (e.g., '345+678'). Maximum length of
# int is DIGITS.
MAXLEN = DIGITS + 1 + DIGITS

# All the numbers, plus sign and space for padding.
chars = '0123456789+ '
ctable = CharacterTable(chars)

questions = []
expected = []
seen = set()
print('Generating data...')
while len(questions) < TRAINING_SIZE:
    f = lambda: int(''.join(np.random.choice(list('0123456789'))
                    for i in range(np.random.randint(1, DIGITS + 1))))
    a, b = f(), f()
    # Skip any addition questions we've already seen
    # Also skip any such that x+Y == Y+x (hence the sorting).
    key = tuple(sorted((a, b)))
    if key in seen:
        continue
    seen.add(key)
    # Pad the data with spaces such that it is always MAXLEN.
    q = '{}+{}'.format(a, b)
    query = q + ' ' * (MAXLEN - len(q))
    ans = str(a + b)
    # Answers can be of maximum size DIGITS + 1.
    ans += ' ' * (DIGITS + 1 - len(ans))
    if REVERSE:
        # Reverse the query, e.g., '12+345  ' becomes '  543+21'. (Note the
        # space used for padding.)
        query = query[::-1]
    questions.append(query)
    expected.append(ans)
print('Total addition questions:', len(questions))

Generating data...
Total addition questions: 50000


In [8]:
questions[0]

'9+0    '

In [9]:
print('Vectorization...')
x = np.zeros((len(questions), MAXLEN, len(chars)), dtype=np.bool)
y = np.zeros((len(questions), DIGITS + 1, len(chars)), dtype=np.bool)
for i, sentence in enumerate(questions):
    x[i] = ctable.encode(sentence, MAXLEN)
for i, sentence in enumerate(expected):
    y[i] = ctable.encode(sentence, DIGITS + 1)


Vectorization...


In [10]:
len(x[0])

7

In [11]:
len(questions[0])

7

In [12]:
questions[0]

'9+0    '

In [13]:
x[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False,  True, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [14]:
y[0]

array([[False, False, False, False, False, False, False, False, False,
        False, False,  True],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False],
       [ True, False, False, False, False, False, False, False, False,
        False, False, False]])

In [15]:
expected[0]

'9   '

In [16]:
# Shuffle (x, y) in unison as the later parts of x will almost all be larger
# digits.
indices = np.arange(len(y))
np.random.shuffle(indices)
x = x[indices]
y = y[indices]

### Step 2: Training/Validation Split

In [17]:
# Explicitly set apart 10% for validation data that we never train over.
split_at = len(x) - len(x) // 10
(x_train, x_val) = x[:split_at], x[split_at:]
(y_train, y_val) = y[:split_at], y[split_at:]

print('Training Data:')
print(x_train.shape)
print(y_train.shape)

print('Validation Data:')
print(x_val.shape)
print(y_val.shape)

Training Data:
(45000, 7, 12)
(45000, 4, 12)
Validation Data:
(5000, 7, 12)
(5000, 4, 12)


### Step 3: Create Model

In [18]:
# input shape: 7 digits, each being 0-9, + or space (12 possibilities)
MAXLEN, len(chars)

(7, 12)

In [19]:
from keras.models import Sequential
from keras import layers

# Try replacing LSTM, GRU, or SimpleRNN.
# RNN = layers.LSTM
RNN = layers.SimpleRNN
# RNN = layers.GRU
HIDDEN_SIZE = 128
BATCH_SIZE = 128

print('Build model...')
model = Sequential()
# encoder: simply encode the same input 4 times 
model.add(RNN(units=HIDDEN_SIZE, input_shape=(MAXLEN, len(chars))))
model.add(layers.RepeatVector(DIGITS + 1))

# decoder: have 4 temporal outputs one for each of the digits of the results
# return_sequences=True tells it to keep all 4 temporal outputs, not only the final one (we need all four digits for the results)
model.add(RNN(units=HIDDEN_SIZE, return_sequences=True))

model.add(layers.Dense(name='classifier', units=len(chars), activation='softmax'))

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
model.summary()

Build model...


Using TensorFlow backend.


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
simple_rnn_1 (SimpleRNN)     (None, 128)               18048     
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 4, 128)            0         
_________________________________________________________________
simple_rnn_2 (SimpleRNN)     (None, 4, 128)            32896     
_________________________________________________________________
classifier (Dense)           (None, 4, 12)             1548      
Total params: 52,492
Trainable params: 52,492
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.predict(np.array([x_val[0]]))

array([[[0.07100666, 0.04503196, 0.12937425, 0.07050125, 0.08865667,
         0.06872862, 0.09661999, 0.10074655, 0.06197888, 0.10655596,
         0.10349934, 0.05729985],
        [0.05981654, 0.0344699 , 0.19359599, 0.07308389, 0.0546543 ,
         0.06734277, 0.11601944, 0.09964471, 0.0401266 , 0.1271986 ,
         0.0755403 , 0.05850698],
        [0.04279239, 0.03352201, 0.18371007, 0.09934117, 0.05451049,
         0.04248012, 0.09836522, 0.05206822, 0.03343712, 0.17121017,
         0.08962404, 0.09893899],
        [0.05258105, 0.03703752, 0.2068308 , 0.10687649, 0.04278126,
         0.02891035, 0.10724873, 0.04250664, 0.02533719, 0.16321443,
         0.0946914 , 0.09198418]]], dtype=float32)

In [21]:
model.predict_classes(np.array([x_val[0]]))

array([[2, 2, 2, 2]], dtype=int64)

### Step 4: Train

In [22]:
# Train the model each generation and show predictions against the validation
# dataset.
for iteration in range(1, 20):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    model.fit(x_train, y_train,
              batch_size=BATCH_SIZE,
              epochs=1,
              validation_data=(x_val, y_val))
    # Select 10 samples from the validation set at random so we can visualize
    # errors.
    for i in range(10):
        ind = np.random.randint(0, len(x_val))
        rowx, rowy = x_val[np.array([ind])], y_val[np.array([ind])]
        preds = model.predict_classes(rowx, verbose=0)
        q = ctable.decode(rowx[0])
        correct = ctable.decode(rowy[0])
        guess = ctable.decode(preds[0], calc_argmax=False)
        print('Q', q[::-1] if REVERSE else q, end=' ')
        print('T', correct, end=' ')
        if correct == guess:
            print(colors.ok + '☑' + colors.close, end=' ')
        else:
            print(colors.fail + '☒' + colors.close, end=' ')
        print(guess)


--------------------------------------------------
Iteration 1
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 9s 199us/step - loss: 1.6761 - acc: 0.3893 - val_loss: 1.5398 - val_acc: 0.4301
Q 281+855 T 1136 ☒ 1068
Q 71+376  T 447  ☒ 313 
Q 161+801 T 962  ☒ 100 
Q 720+54  T 774  ☒ 803 
Q 30+207  T 237  ☒ 300 
Q 355+6   T 361  ☒ 410 
Q 3+649   T 652  ☒ 441 
Q 86+606  T 692  ☒ 774 
Q 7+320   T 327  ☒ 333 
Q 41+929  T 970  ☒ 900 

--------------------------------------------------
Iteration 2
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 6s 124us/step - loss: 1.4452 - acc: 0.4654 - val_loss: 1.3477 - val_acc: 0.4991
Q 930+47  T 977  ☒ 902 
Q 46+46   T 92   ☒ 11  
Q 11+958  T 969  ☒ 990 
Q 424+872 T 1296 ☒ 1334
Q 200+222 T 422  ☒ 322 
Q 55+4    T 59   ☒ 58  
Q 165+655 T 820  ☒ 102 
Q 3+928   T 931  ☒ 922 
Q 513+22  T 535  ☒ 565 
Q 35+360  T 395  ☒ 482 

-------------------

45000/45000 [==============================] - 4s 79us/step - loss: 0.2781 - acc: 0.9057 - val_loss: 0.2907 - val_acc: 0.8996
Q 2+89    T 91   ☒ 80  
Q 86+727  T 813  ☑ 813 
Q 33+44   T 77   ☑ 77  
Q 865+5   T 870  ☑ 870 
Q 646+16  T 662  ☒ 672 
Q 176+67  T 243  ☒ 233 
Q 746+684 T 1430 ☒ 1530
Q 86+25   T 111  ☒ 112 
Q 151+44  T 195  ☑ 195 
Q 97+16   T 113  ☒ 114 

--------------------------------------------------
Iteration 16
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [==============================] - 4s 79us/step - loss: 0.2605 - acc: 0.9116 - val_loss: 0.2798 - val_acc: 0.9019
Q 466+495 T 961  ☒ 951 
Q 596+479 T 1075 ☒ 1065
Q 0+208   T 208  ☑ 208 
Q 15+926  T 941  ☒ 930 
Q 197+293 T 490  ☑ 490 
Q 45+41   T 86   ☑ 86  
Q 95+504  T 599  ☒ 609 
Q 161+9   T 170  ☒ 160 
Q 316+38  T 354  ☑ 354 
Q 94+643  T 737  ☑ 737 

--------------------------------------------------
Iteration 17
Train on 45000 samples, validate on 5000 samples
Epoch 1/1
45000/45000 [=======

In [23]:
x_val[0]

array([[False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False,  True, False, False,
        False, False, False],
       [False,  True, False, False, False, False, False, False, False,
        False, False, False],
       [False, False, False, False, False,  True, False, False, False,
        False, False, False],
       [False, False, False, False, False, False, False,  True, False,
        False, False, False],
       [False, False, False, False, False, False, False, False, False,
        False, False,  True]])

In [24]:
model.predict(np.array([x_val[0]]))

array([[[2.7147005e-06, 1.4477567e-07, 8.4531749e-08, 8.2284708e-07,
         2.4795764e-07, 5.3130930e-06, 3.7707055e-03, 3.0721015e-01,
         6.0830373e-01, 7.8974240e-02, 1.7248653e-03, 6.9469625e-06],
        [3.5350616e-09, 8.8691732e-10, 3.8329381e-02, 7.3532210e-05,
         8.7248523e-08, 1.0520136e-09, 5.2830912e-10, 2.4600881e-09,
         6.6752028e-07, 3.3753249e-04, 5.6708239e-02, 9.0455061e-01],
        [2.1649005e-04, 2.8848893e-10, 5.6022964e-07, 2.7527494e-04,
         1.5076913e-01, 8.4345990e-01, 5.2751880e-03, 3.4669583e-06,
         4.9487405e-09, 2.8662218e-12, 3.0233506e-13, 7.1515599e-11],
        [9.9996626e-01, 1.0162692e-12, 8.5525575e-13, 1.7106087e-10,
         4.7913411e-09, 5.6313735e-07, 3.0312738e-05, 2.8597522e-06,
         3.6204748e-10, 1.5683542e-14, 9.2559360e-17, 1.3130477e-15]]],
      dtype=float32)

In [25]:
model.predict_classes(np.array([x_val[0]]))

array([[ 8, 11,  5,  0]], dtype=int64)